In [ ]:
#Notes for Lucy

#The first code is a first look at sentinel 2 image and ....sentinel1?
#3rd code?
#Create in GITHUB a read me file... (include comentaries about the class and the importance of this code for the study)

In [ ]:
# @title INSTALL LIBRARIES (run this if is your first time running)
!pip install rasterio numpy pillow matplotlib -q

In [ ]:
# @title RUN LIBRARIES AND MOUNTAIN YOUR GOOGLE DRIVE

import os, glob, numpy as np, rasterio
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title FUNCTIONS

class SentinelToPNGConverter:
    """
    A class to convert Sentinel-2 GeoTIFF images to enhanced RGB PNG images.
    Applies normalization, contrast, brightness, and gamma corrections.
    """
    def __init__(self, input_dir, output_dir, gamma=1.2, brightness=1.1, contrast=1.3):
        """
        Initialize the converter with input/output directories and enhancement parameters.

        Parameters:
        - input_dir: Directory containing input GeoTIFF images.
        - output_dir: Directory to save the output PNG images.
        - gamma: Gamma correction factor.
        - brightness: Brightness adjustment factor.
        - contrast: Contrast adjustment factor.
        """
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.gamma = gamma
        self.brightness = brightness
        self.contrast = contrast
        os.makedirs(self.output_dir, exist_ok=True)

    def stretch(self, arr):
        """
        Normalize the pixel values to the range [0, 1] using fixed min and max values.
        This is a linear stretch assuming reflectance values from 0.0 to 0.02.
        """
        return np.clip((arr - 0.0) / (0.02 - 0.0), 0, 1)

    def get_band_indices(self, descriptions):
        """
        Get the indices for bands B4 (Red), B3 (Green), and B2 (Blue).
        If band names are not found, fallback to default band positions 1, 2, 3.

        Parameters:
        - descriptions: List of band descriptions from the TIFF metadata.

        Returns:
        - Tuple of indices (B4, B3, B2)
        """
        desc = [d.upper() if d else '' for d in descriptions]
        idx_B4 = desc.index('B4') + 1 if 'B4' in desc else 1
        idx_B3 = desc.index('B3') + 1 if 'B3' in desc else 2
        idx_B2 = desc.index('B2') + 1 if 'B2' in desc else 3
        return idx_B4, idx_B3, idx_B2

    def process_image(self, tif_path):
        """
        Process a single GeoTIFF file and save the enhanced RGB image as PNG.

        Steps:
        - Read bands B4, B3, B2
        - Normalize and stack as RGB
        - Apply brightness, contrast, and gamma adjustments
        - Save as PNG
        """
        png_name = os.path.splitext(os.path.basename(tif_path))[0] + '.png'
        out_path = os.path.join(self.output_dir, png_name)

        with rasterio.open(tif_path) as src:
            idx_B4, idx_B3, idx_B2 = self.get_band_indices(src.descriptions)

            rgb = np.dstack([
                self.stretch(src.read(idx_B4).astype(np.float32)),
                self.stretch(src.read(idx_B3).astype(np.float32)),
                self.stretch(src.read(idx_B2).astype(np.float32))
            ])

        # Apply contrast, brightness, and gamma corrections
        rgb = self.contrast * (rgb - 0.5) + 0.5
        rgb *= self.brightness
        rgb = np.power(np.clip(rgb, 0, 1), self.gamma)

        # Save the result as an 8-bit RGB PNG image
        Image.fromarray((rgb * 255).astype(np.uint8), 'RGB').save(out_path)

    def convert_all(self):
        """
        Convert all GeoTIFF files in the input directory to enhanced RGB PNG images.
        """
        tifs = glob.glob(os.path.join(self.input_dir, '*.tif'))
        for tif in tifs:
            self.process_image(tif)
        print(f'✅  Converted {len(tifs)} images to PNG RGB in:\n   {self.output_dir}')

In [ ]:
if __name__ == "__main__":
    # Create the converter object and start the batch conversion
    converter = SentinelToPNGConverter(
        input_dir='/content/drive/MyDrive/GEE_Exports/SENTINEL', #changes according to your folders in drive
        output_dir='/content/drive/MyDrive/GEE_Exports/PNG' #changes according to your folders in drive
    )
    converter.convert_all()